Carteira baseada no número de Graham sobre o valor intrínseco de uma ação, foca em pegar empresas que sejam lucrativas e baratas: empresas operando a P/L menor que 15 e com P/VPA menor que 1,5. Multiplicando 15 x 1,5 temos o número 22,5 que nos indica a fórmula de graham. Valor Intrínseco de uma ação = $\sqrt{22,5 \times \text{LPA} \times \text{VPA}}$.

Critérios:


- Ter Lucro por ação maior que zero ,isto é, a empresa não pode estar com prejuízo atualmente
- Ter Valor Patrimonial por ação positivo, ou seja, a empresa não pode ter mais passivos (obrigações a pagar) que ativos (bens ou direitos a receber)
- Ter volume médio de negociação diário de no mínimo R$250.000,00
- Ter lucro líquido medio positivo em todos os últimos 5 exercícios.


In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True): #beauty print :)
    
    # from IPython.display import display, HTML

    # if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
    #     df['prefixo'] = df['Papel'].astype(str).str[:4]
    #     df=df.drop_duplicates(subset='prefixo', keep='first')
    #     # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [4]:
from DT_atualiza_settings import *
from DT_StatusInvest import SI
SI(mercado = 'Acoes' )

 ====== SI Acoes ===== 
 ====== Busca
 ====== Download 


In [26]:
import os
# Caminho do arquivo local
file_path = os.path.expanduser('~/GHub/Finance-playground/data/SI_Acoes.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Acoes.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [27]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [28]:
fundsSI =  funds[ (funds[' LIQUIDEZ MEDIA DIARIA'] > 300000) &
                  (funds[' LPA'] > 0) & 
                  (funds[' VPA'] > 0) & 
                  (funds['CAGR LUCROS 5 ANOS'] > 0) ]
fundsSI

fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,'LIQUIDEZ MEDIA DIARIA (em M)',
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
LIGT3,6.50,12.76,6.655030,15.14,6.50,2.421610,47.06,624.00,1.00,3.87
SYNE3,6.07,76.27,2.187578,7.15,3.66,0.926552,24.27,299.84,1.66,0.48
JHSF3,5.24,27.06,10.344268,8.44,1.57,3.570131,17.27,229.58,3.35,2.09
BAZA3,74.64,64.26,0.612682,115.26,23.27,4.184193,245.66,229.13,3.21,NaN
BRAP3,15.10,8.66,0.709916,21.50,4.59,6.241887,47.12,212.05,3.29,-0.17
LOGG3,20.40,32.94,3.591800,41.80,4.26,1.792327,63.30,210.29,4.79,3.59
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
COGN3,2.87,33.12,67.565578,6.63,0.52,5.385860,8.81,206.97,5.48,3.83
LVTC3,3.59,9.93,0.495618,10.45,0.51,0.227827,10.95,205.01,7.10,-2.10
JBSS3,39.03,12.41,3698.504235,49.88,12.55,33.886011,118.68,204.07,3.11,3.33


outros criterios adicionais

P/L > 0 e DIVIDA LIQUIDA / EBIT < 3

In [29]:
'''
P/L > 0 e DIVIDA LIQUIDA / EBIT < 3
'''
fundsSI.fillna(0,inplace=True) ## bancos tem NaN
b_print(fundsSI[(fundsSI['P/L'] > 0) & (fundsSI['DIVIDA LIQUIDA / EBIT'] < 3) ][colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
SYNE3,6.07,76.27,2.187578,7.15,3.66,0.926552,24.27,299.84,1.66,0.48
JHSF3,5.24,27.06,10.344268,8.44,1.57,3.570131,17.27,229.58,3.35,2.09
BAZA3,74.64,64.26,0.612682,115.26,23.27,4.184193,245.66,229.13,3.21,0.00
BRAP3,15.10,8.66,0.709916,21.50,4.59,6.241887,47.12,212.05,3.29,-0.17
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
LVTC3,3.59,9.93,0.495618,10.45,0.51,0.227827,10.95,205.01,7.10,-2.10
BRAP4,16.30,8.66,51.848648,21.50,4.59,6.241887,47.12,189.08,3.55,-0.17
ISAE4,23.14,15.15,27.554698,30.98,5.42,17.201728,61.47,165.64,4.27,2.20
BBAS3,22.08,8.51,556.979494,31.72,4.30,126.536816,55.40,150.91,5.13,0.00
GOAU4,9.40,21.31,55.977397,19.58,1.12,9.304323,22.21,136.28,8.36,1.25


remove empresas repetidas, mantem primeira ocorrencia

In [30]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
b_print(df_limpo[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
LIGT3,6.50,12.76,6.655030,15.14,6.50,2.421610,47.06,624.00,1.00,3.87
SYNE3,6.07,76.27,2.187578,7.15,3.66,0.926552,24.27,299.84,1.66,0.48
JHSF3,5.24,27.06,10.344268,8.44,1.57,3.570131,17.27,229.58,3.35,2.09
BAZA3,74.64,64.26,0.612682,115.26,23.27,4.184193,245.66,229.13,3.21,0.00
BRAP3,15.10,8.66,0.709916,21.50,4.59,6.241887,47.12,212.05,3.29,-0.17
LOGG3,20.40,32.94,3.591800,41.80,4.26,1.792327,63.30,210.29,4.79,3.59
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
COGN3,2.87,33.12,67.565578,6.63,0.52,5.385860,8.81,206.97,5.48,3.83
LVTC3,3.59,9.93,0.495618,10.45,0.51,0.227827,10.95,205.01,7.10,-2.10
JBSS3,39.03,12.41,3698.504235,49.88,12.55,33.886011,118.68,204.07,3.11,3.33


### outra estória de porque 22,5

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce. Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos. O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores. Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

# Stocks

In [31]:
import os
# Caminho do arquivo local /home/yair/GHub/Codigos-em-financas/data/SI_Stocks.csv
file_path = os.path.expanduser('/home/yair/GHub/Codigos-em-financas/data/SI_Stocks.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Stocks.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido da URL.


In [32]:
funds

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
0,A,118.06,0.82,28.68,5.45,2.75,53.35,22.36,17.59,22.57,23.92,1.36,0.33,5.05,15.23,-4.21,2.09,19.00,9.59,13.26,0.50,0.50,0.55,4.75,3.77,NaN,21.60,4.10,-4.69,3.344294e+10
1,AA,28.82,1.05,8.62,1.28,0.51,20.43,11.56,6.79,5.06,6.76,1.69,0.43,0.59,3.42,-0.79,1.71,14.79,5.90,11.09,0.40,0.59,0.87,2.66,NaN,NaN,22.46,3.32,-0.04,7.414962e+09
2,AACG,0.84,NaN,-2.56,1.16,0.37,51.68,-31.49,-32.34,-2.63,-2.02,0.61,-0.27,0.83,-0.79,-0.43,0.23,-45.26,-14.39,-39.89,0.32,0.68,0.44,21.21,NaN,NaN,0.72,-0.33,-0.06,2.656458e+07
3,AACQ,9.99,NaN,-93.43,180.94,1.25,NaN,NaN,NaN,-27.75,-27.74,0.01,-0.07,NaN,2094.26,-1.25,3.56,-193.66,-1.33,-652.08,0.01,0.12,0.00,NaN,NaN,NaN,0.06,-0.11,NaN,9.047194e+08
4,AACQU,11.05,NaN,-103.35,200.14,1.38,NaN,NaN,NaN,-30.69,-27.74,0.01,-0.07,NaN,2316.47,-1.38,3.56,-193.66,-1.33,-652.08,0.01,0.12,0.00,NaN,NaN,NaN,0.06,-0.11,NaN,9.047194e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5247,ZUO,10.02,NaN,-20.82,8.36,1.80,67.42,-10.54,-16.34,-32.28,-28.30,3.99,-1.03,3.40,3.65,-8.80,2.62,-40.16,-8.63,-8.89,0.21,0.79,0.53,349.59,NaN,NaN,1.20,-0.48,0.40,1.540242e+09
5248,ZVO,0.09,NaN,-0.08,0.47,0.04,35.09,-17.75,-17.80,-0.08,0.59,0.66,-4.11,0.01,-1.38,-0.07,0.95,-619.73,-47.91,-619.73,0.08,0.92,2.69,-12.86,NaN,NaN,0.19,-1.20,0.00,3.078549e+06
5249,ZYME,12.64,NaN,-7.73,2.70,2.07,100.00,-114.77,-121.73,-8.20,-5.72,2.48,-0.82,9.41,3.64,-7.38,4.71,-34.98,-26.71,-34.98,0.76,0.24,0.22,20.90,NaN,NaN,4.66,-1.63,0.79,8.787089e+08
5250,ZYNE,1.30,NaN,-1.87,0.00,0.00,NaN,NaN,NaN,-1.87,955.19,957.05,-1.24,NaN,0.00,-0.06,3.43,-0.13,-0.09,-0.13,0.72,0.28,0.00,NaN,NaN,NaN,538.03,-0.70,4.43,7.012122e+07


In [33]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [34]:
fundsSI =  funds[ #(funds[' LIQUIDEZ MEDIA DIARIA'] > 300000) &
                  (funds[' LPA'] > 0) & 
                  (fundsSI['DIVIDA LIQUIDA / EBIT'] < 3) & 
                  (funds[' VPA'] > 0) & 
                  (funds['CAGR LUCROS 5 ANOS'] > 0) ]
b_print(fundsSI,100)

TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
AAPL,198.54,0.51,30.30,44.14,8.90,46.63,31.72,24.30,23.21,23.61,0.39,0.74,7.36,-113.84,-13.87,0.82,145.66,29.37,58.97,0.20,0.80,1.21,8.49,11.15,NaN,4.47,6.51,0.44,2.948183e+12
ABB,37.12,NaN,17.32,5.68,1.80,32.37,17.56,13.65,13.46,13.66,0.20,0.09,2.36,89.85,-3.35,1.04,32.76,10.36,24.67,0.32,0.68,0.76,3.17,2.26,NaN,6.54,2.14,0.15,6.900207e+10
ABC,179.95,NaN,21.39,52.69,0.59,3.42,0.91,0.66,15.60,17.17,1.57,5.29,0.14,-6.24,-1.79,0.88,246.27,2.76,33.33,0.01,0.99,4.16,9.27,36.05,NaN,3.42,8.41,-2.59,3.614368e+10
ABCB,61.20,1.15,11.30,1.10,0.16,0.00,31.82,22.24,7.90,2.02,-5.88,-0.82,2.51,NaN,-0.16,NaN,9.74,1.40,10.09,0.14,0.86,0.06,14.90,17.31,NaN,55.49,5.40,0.36,4.206321e+09
ABG,236.36,NaN,11.15,1.28,0.45,17.06,4.79,2.42,5.65,11.09,5.44,1.23,0.27,7.24,-0.66,1.25,11.46,4.06,8.28,0.35,0.65,1.68,18.98,18.47,NaN,184.32,21.12,-0.41,4.630725e+09
ABT,136.36,0.81,17.54,4.85,2.91,55.81,18.67,31.89,29.97,30.77,0.81,0.13,5.59,23.34,-4.06,1.78,27.66,16.58,2.83,0.60,0.40,0.52,5.63,29.45,NaN,28.05,7.76,0.04,2.368440e+11
ABTX,41.63,NaN,10.98,1.18,0.12,0.00,38.18,28.90,8.31,-11.75,-20.06,-2.84,3.17,NaN,-0.12,NaN,10.73,1.12,11.72,0.10,0.90,0.04,19.36,28.98,NaN,35.34,3.79,3.01,8.309431e+08
AC,38.87,5.40,21.52,0.91,0.88,100.00,380.10,316.45,17.92,11.51,-6.41,-0.33,68.11,NaN,-0.88,NaN,4.25,4.09,4.29,0.96,0.03,0.01,-16.10,2.50,NaN,42.53,1.81,1.55,8.216224e+08
ACGL,91.82,5.47,9.20,1.59,0.46,33.91,22.72,20.50,8.31,8.78,0.48,0.09,1.89,NaN,-0.46,NaN,17.29,4.96,13.78,0.29,0.71,0.24,20.28,22.01,NaN,57.49,9.94,0.18,3.428629e+10
ACGLO,20.26,5.05,2.03,0.35,0.10,33.91,22.72,20.50,1.84,8.78,0.48,0.09,0.42,NaN,-0.10,NaN,17.29,4.96,13.78,0.29,0.71,0.24,20.28,22.01,NaN,57.49,9.94,0.04,3.428629e+10


In [35]:
fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
# fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
ACGLO,20.26,22.01,57.49,9.94,34.286289,113.39,459.67,2.03,0.48
ACGLP,24.99,22.01,57.49,9.94,34.286289,113.39,353.74,2.51,0.48
ACHC,21.28,18.60,33.02,2.04,1.964045,38.93,82.94,10.47,5.63
AEPPL,52.61,9.08,51.14,5.17,54.941956,77.13,46.61,10.17,9.55
ABCB,61.20,17.31,55.49,5.40,4.206321,82.11,34.17,11.30,-5.88
ABTX,41.63,28.98,35.34,3.79,0.830943,54.90,31.88,10.98,-20.06
ABG,236.36,18.47,184.32,21.12,4.630725,295.95,25.21,11.15,5.44
ACGL,91.82,22.01,57.49,9.94,34.286289,113.39,23.49,9.20,0.48
AES,11.65,40.84,4.87,1.82,8.236947,14.12,21.20,6.37,13.42
AFBI,18.22,72.62,19.36,0.94,0.115079,20.24,11.09,19.38,-9.49
